***This is Coursera Capstone Week3 Assignment:***

<h2 align=center> Segmenting and Clustering Neighborhoods in Toronto</h2>

Imporing the essesntail library for the task. Since, we need to create dataframe, get the geocode of location and finally map those, we will need atleast of these following libraries:

In [1]:
import pandas as pd #importing pandas for scarpping data from URL(wikipedia page)
import geopy.geocoders#importing gecode library for location (latitude and longitude) mapping
import folium # importing folium for map
import requests
from pandas.io.json import json_normalize

**Scrapping data from the website**

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'#given assignment website
df = pd.read_html(url, attrs={"class": "wikitable sortable"})[0] # 0 is for the 1st table in this particular page
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [3]:
df_copy=df.copy()# making copy for original data to have a raw data backups for validations and checks, if needed.

**Removing all the Borough with values "Not Assigned" by updating df.**

In [4]:
df=df[df.Borough != "Not assigned"]
df.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
#checking that df is updated c]orrectly
raw=(df_copy.Borough == "Not assigned").sum()
new=((df.Borough == "Not assigned").sum())
print(raw,"vs", new)

77 vs 0


<b>There are <u>no Borough with Not assigned Neighborhood</u>. As all not assigned neighbourhood got removed with Borough with not assigned. Following check is done to confirm</b>

In [6]:
raw=(df_copy.Neighbourhood=="Not assigned").sum()
new=(df.Neighbourhood=="Not assigned").sum()
print(raw,'vs', new)

77 vs 0


<h3><b>Getting Geocodes</b></h3>

(Tried using geocoder.google but it resulted in empty values, hence switched to different way of extracting the codes.)


In [7]:
# ArcGIS is part of geopy.geocoders
nom=geopy.geocoders.ArcGIS()
df["LOCATION"]=df["Postal Code"]+", Toronto, Ontario" #combining the complete address for location
df["Coordinates"]=df["LOCATION"].apply(nom.geocode)#getting the gecode of the location
df["Latitude"]=df["Coordinates"].apply(lambda x: x.latitude)#mapping latitute
df["Longitude"]=df["Coordinates"].apply(lambda x: x.longitude)#mapping longitude

In [8]:
df.drop(columns=['LOCATION','Coordinates'],inplace=True)#removing extra columns

**The final dataframe as required by the task**

In [9]:
df.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.75245,-79.32991
3,M4A,North York,Victoria Village,43.73057,-79.31306
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.66263,-79.52831
9,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
11,M3B,North York,Don Mills,43.74923,-79.36186
12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804


<h3><b>Maping geocodes on the map using folium for initial exploratory analysis and see what areas to cluster</b></h3>

In [10]:
# create map of Toronto using latitude and longitude values
Toronto=nom.geocode("Toronto, Ontario")[1]
Map_Toronto = folium.Map(location=[Toronto[0],Toronto[1]], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='white',
        fill_opacity=0.7,
        parse_html=False).add_to(Map_Toronto)
    
display(Map_Toronto)  

We can see that most of the Borough are concentated in Downtown Toronto. However, the value counts of Borough of North York is 24 (highest) and Downtown is 19. This is more because the North York is much bigger Borough than Downtown. 


In [11]:
df.Borough.value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

<h2><b>Now will be exploring how many Indian places are there in Downtown Area</b></h2>

In [12]:
CLIENT_ID = 'MLSV155ET5DBXXXXXXXXXXXXXXXXXXXXXDEZF5YXH1VCA4Y' # Foursquare ID
CLIENT_SECRET = 'ELPUUXXXXXXXXXXXXXXXXXXXXXXXXXXXXXIIBPD3O' # Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20201231'
LIMIT = 1000


In [13]:
#getting the geocodes of Downtown
address = 'Downtown Toronto, Toronto,Ontario,Canada'
geolocator =geopy.geocoders.Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6563221 -79.3809161


In [14]:
#Searching for venues with 'Indian' in name.
search_query ='Indian'
radius ='3000'

In [15]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6011b5e90860021febc3d3b2'},
 'response': {'venues': [{'id': '4aef8854f964a5201cd921e3',
    'name': 'Aroma Fine Indian Restaurant',
    'location': {'address': '287 King St. W',
     'crossStreet': 'at John St.',
     'lat': 43.646462521503445,
     'lng': -79.38964414801342,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.646462521503445,
       'lng': -79.38964414801342}],
     'distance': 1303,
     'postalCode': 'M5V 1J5',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['287 King St. W (at John St.)',
      'Toronto ON M5V 1J5',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'shortName': 'Indian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1611773417',
  

In [17]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
DT= json_normalize(venues)
DT.head()

<ipython-input-17-848cecce0461>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  DT= json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,4aef8854f964a5201cd921e3,Aroma Fine Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1611773417,False,287 King St. W,at John St.,43.646463,-79.389644,"[{'label': 'display', 'lat': 43.64646252150344...",1303,M5V 1J5,CA,Toronto,ON,Canada,"[287 King St. W (at John St.), Toronto ON M5V ...",NaN,NaN
1,4ad4c060f964a5206ff720e3,309 Dhaba Indian Excellence,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1611773417,False,309 King Street West,NaN,43.646394,-79.390418,"[{'label': 'display', 'lat': 43.64639421226189...",1344,M5V 1J5,CA,Toronto,ON,Canada,"[309 King Street West, Toronto ON M5V 1J5, Can...",NaN,NaN
2,4b2a634af964a52020a824e3,Indian Flavour,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1611773417,False,123 Dundas St W,btw Elizabeth & Bay,43.655649,-79.384119,"[{'label': 'display', 'lat': 43.65564910619165...",268,NaN,CA,Toronto,ON,Canada,"[123 Dundas St W (btw Elizabeth & Bay), Toront...",NaN,NaN
3,4afd920ff964a520ad2822e3,Indian Biriyani House,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1611773417,False,181 Dundas St W,W of Chestnut St,43.655120,-79.386645,"[{'label': 'display', 'lat': 43.65511996683289...",480,M5G 1C7,CA,Toronto,ON,Canada,"[181 Dundas St W (W of Chestnut St), Toronto O...",NaN,NaN
4,5165c333e4b07a7ad88d8a69,Joe's Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1611773417,False,NaN,NaN,43.658150,-79.381563,"[{'label': 'display', 'lat': 43.65814977325445...",210,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN


In [18]:
DT.drop(columns=['id',  'categories', 'referralId', 'hasPerk','location.labeledLatLngs','location.cc', 'location.city',
       'location.state', 'location.country','venuePage.id'], inplace=True)
DT.head()

,name,location.address,location.crossStreet,location.lat,location.lng,location.distance,location.postalCode,location.formattedAddress,location.neighborhood
0,Aroma Fine Indian Restaurant,287 King St. W,at John St.,43.646463,-79.389644,1303,M5V 1J5,"[287 King St. W (at John St.), Toronto ON M5V ...",NaN
1,309 Dhaba Indian Excellence,309 King Street West,NaN,43.646394,-79.390418,1344,M5V 1J5,"[309 King Street West, Toronto ON M5V 1J5, Can...",NaN
2,Indian Flavour,123 Dundas St W,btw Elizabeth & Bay,43.655649,-79.384119,268,NaN,"[123 Dundas St W (btw Elizabeth & Bay), Toront...",NaN
3,Indian Biriyani House,181 Dundas St W,W of Chestnut St,43.655120,-79.386645,480,M5G 1C7,"[181 Dundas St W (W of Chestnut St), Toronto O...",NaN
4,Joe's Indian Restaurant,NaN,NaN,43.658150,-79.381563,210,NaN,[Canada],NaN


In [19]:
#Creating a Folium map to mark the Indina Places
Toronto=nom.geocode("Toronto, Ontario")[1]
Map_Toronto = folium.Map(location=[Toronto[0],Toronto[1]], zoom_start=14)

# add markers to map
for lat, lng, name, address in zip(DT['location.lat'], DT['location.lng'], DT['name'], DT['location.formattedAddress']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='brown',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(Map_Toronto)
    
display(Map_Toronto)  